In [ ]:
import pandas as pd
import io
from google.colab import files

# ==============================================================================
# Country-Year Panel Construction from Multiple Public CSV Tables
# ------------------------------------------------------------------------------
# This script uploads multiple CSV source files, extracts metadata, builds a
# complete country-year skeleton, and merges all indicator tables into a single
# harmonized panel dataset for downstream MA-TabNet experiments.
# ==============================================================================

In [ ]:
# 1. Upload source files
print("Please click the [Choose Files] button below and upload all 7 CSV source files.")
uploaded = files.upload()

# Retrieve uploaded file names
file_paths = list(uploaded.keys())
print(f"\nA total of {len(file_paths)} files were uploaded: {file_paths}")

dfs = []
meta_list = []

print("\n--- Starting data loading and deterministic merging pipeline ---")

for f in file_paths:
    try:
        # Read uploaded CSV content
        df = pd.read_csv(io.BytesIO(uploaded[f]))

        # Standardize column names by stripping leading/trailing whitespace
        df.columns = df.columns.str.strip()

        # Extract metadata columns when available
        meta_cols = ['Country Code', 'Country Name']
        if 'Continent' in df.columns:
            meta_cols.append('Continent')

        # Store metadata candidates for later restoration
        if 'Country Code' in df.columns:
            cols_to_use = [c for c in meta_cols if c in df.columns]
            meta_list.append(df[cols_to_use].drop_duplicates())

        dfs.append(df)
        print(f"Processed successfully: {f}")

    except Exception as e:
        print(f"[Error] Failed while processing {f}: {e}")

In [ ]:
# 2. Build a country-year skeleton
# This ensures that all available country-year combinations appearing in the
# source files are retained before feature-level merging.
skeleton = pd.DataFrame()
for df in dfs:
    if 'Country Code' in df.columns and 'Year' in df.columns:
        skeleton = pd.concat([skeleton, df[['Country Code', 'Year']]])

skeleton = (
    skeleton
    .drop_duplicates()
    .sort_values(['Country Code', 'Year'])
    .reset_index(drop=True)
)

In [ ]:
# 3. Merge indicator tables onto the skeleton
merged_df = skeleton.copy()

for df in dfs:
    keys = ['Country Code', 'Year']
    meta_cols = ['Country Name', 'Continent']
    value_cols = [c for c in df.columns if c not in keys and c not in meta_cols]

    # Skip tables that do not contain indicator/value columns
    if not value_cols:
        continue

    subset = df[keys + value_cols]

    # Remove duplicated country-year rows to preserve deterministic merging
    if subset.duplicated(subset=keys).any():
        subset = subset.drop_duplicates(subset=keys)

    merged_df = pd.merge(merged_df, subset, on=keys, how='left')

In [ ]:
# 4. Restore metadata
# Metadata are consolidated across files and reattached after merging.
all_meta = pd.concat(meta_list, ignore_index=True)

if 'Continent' in all_meta.columns:
    all_meta = all_meta.sort_values('Continent', na_position='last')

unique_meta = all_meta.drop_duplicates(subset=['Country Code'], keep='first')
final_df = pd.merge(unique_meta, merged_df, on='Country Code', how='right')

# Reorder columns for readability
cols = ['Country Code', 'Country Name']
if 'Continent' in final_df.columns:
    cols.append('Continent')
cols.append('Year')

remaining_cols = [c for c in final_df.columns if c not in cols]
final_df = final_df[cols + remaining_cols]

In [ ]:
# 5. Save and download the merged country-year panel
output_filename = 'global_mmr_country_year_panel_v1.csv'
final_df.to_csv(output_filename, index=False)

print(f"\n[Completed] The merged country-year panel has been saved as: {output_filename}")
print("The file will now be downloaded automatically.")
files.download(output_filename)